<div class="alert alert-block alert-success">
<b>Bibliotecas importadas</b>
</div>

In [1]:
import numpy as np
import chart_studio
import pandas as pd
import plotly.graph_objs as go
import chart_studio.plotly as py

chart_studio.tools.set_config_file(world_readable = True, sharing = 'public')

# Change default pandas display options
pd.options.display.float_format = '{:,.2f}'.format

<div class="alert alert-block alert-info">
<b>Read the excel file</b>
</div>

In [2]:
df = pd.read_excel('Cestas.xlsx', sheet_name = 'Data frame')
df.set_index(['Alimento'], inplace = True) 
df.sort_index(inplace = True)
df.head()

,Faixa etária,Quantidade de alunos,Quantidade/aluno,kg/unidade,Preço/unidade,kg/aluno,Preço/aluno,kg total,Preço total
Alimento,,,,,,,,,
Arroz,6 meses a 3 anos,1998,1,1.00,2.89,1.00,2.89,"1,998.00","5,774.22"
Arroz,3 anos a 6 anos,2158,2,1.00,2.89,2.00,5.78,"4,316.00","12,473.24"
Arroz,6 anos a 15 anos,9691,2,1.00,2.89,2.00,5.78,"19,382.00","56,013.98"
Açucar,3 anos a 6 anos,2158,1,1.00,2.29,1.00,2.29,"2,158.00","4,941.82"
Açucar,6 anos a 15 anos,9691,1,1.00,2.29,1.00,2.29,"9,691.00","22,192.39"


<div class="alert alert-block alert-info">
<b>Pivot table</b>
</div>

In [3]:
pivot_all = pd.pivot_table(df, index = ['Alimento'], 
                               columns = ['Faixa etária'],
                               values = ['kg total'], 
                               aggfunc = np.sum, fill_value = 0) 
# Sort by alimento
pivot_all = pivot_all.sort_values(by = ['Alimento'], ascending = True)

pivot_all.head(9)

kg total                                  
Faixa etária        3 anos a 6 anos 6 anos a 15 anos 6 meses a 3 anos
Alimento                                                             
Arroz                      4,316.00        19,382.00         1,998.00
Açucar                     2,158.00         9,691.00             0.00
Biscoito cracker             863.20         3,876.40             0.00
Biscoito maria               863.20         3,876.40             0.00
Charque traseiro               0.00         4,845.50             0.00
Farinha de mandioca            0.00         9,691.00             0.00
Feijão                     2,158.00         9,691.00         1,998.00
Leite                      1,726.40             0.00         1,598.40
Macarrão                   2,158.00         9,691.00           999.00

<div class="alert alert-block alert-info">
<b>Limited pivot table</b>
</div>

In [4]:
pivot_one = pd.pivot_table(df, index = ['Alimento', 'Faixa etária'], 
                               values = ['kg total', 'Preço total'], # Doesn't follow the order
                               aggfunc = 'sum', fill_value = 0) 

# Select data that will show on dataframe and change the index/row order
pivot_one = pivot_one.reindex([('Leite', '6 meses a 3 anos'),
                               ('Feijão', '6 meses a 3 anos'),
                               ('Macarrão', '6 meses a 3 anos')])
                               # axis = 0 (row) is the default
# Sort by alimento
pivot_one = pivot_one.sort_values(by = ['Alimento'], ascending = True)

# Reorder the values of pivot table
pivot_one = pivot_one[['kg total', 'Preço total']]

# You can rename the values if needed but don't do it like this
# pivot_one.columns = ['Preço total', 'kg total']
# Instead use:
# pivot_one = pivot_one.rename(columns = {'Preço total': 'something'})

pivot_one

,,kg total,Preço total
Alimento,Faixa etária,,
Feijão,6 meses a 3 anos,"1,998.00","12,567.42"
Leite,6 meses a 3 anos,"1,598.40","37,482.48"
Macarrão,6 meses a 3 anos,999.00,"3,976.02"


In [5]:
pivot_two = pd.pivot_table(df, index = ['Alimento'],
                               columns = ['Faixa etária'],
                               values = ['kg total', 'Preço total'], # Doesn't follow the order
                               aggfunc = np.sum, fill_value = 0) 

# Sort by preço total with 6 meses a 3 anos
pivot_two = pivot_two.sort_values(by = [('Preço total', '6 anos a 15 anos')], ascending = False)

# Reorder the values of pivot table
pivot_two = pivot_two[['kg total', 'Preço total']]

<div class="alert alert-block alert-info">
<b>Multi-level limited pivot table</b>
</div>

In [6]:
pivot_three = pd.pivot_table(df, index = ['Alimento'],
                               columns = ['Faixa etária'],
                               values = ['kg total', 'Preço total'], # Doesn't follow the order
                               aggfunc = np.sum, fill_value = 0) 

# Select data that will show on dataframe and change the value/column order
pivot_three = pivot_three.reindex([('Preço total', '6 anos a 15 anos'),
                               ('kg total', '6 anos a 15 anos')], axis = 1)

# Sort by preço total with 6 meses a 3 anos
pivot_three = pivot_three.sort_values(by = [('Preço total', '6 anos a 15 anos')], ascending = False)

# Reorder the values of pivot table
pivot_three = pivot_three[['kg total', 'Preço total']]

<div class="alert alert-block alert-info">
<b>Working with a series</b>
</div>

In [7]:
df0 = pivot_all['kg total']['3 anos a 6 anos'] # Pandas series
df0 = df0.to_frame() # Transform into a dataframe so it's possible to use reset_index()
# could also use df1 = pd.DataFrame(pivot_all['kg total']['3 anos a 6 anos'])
df0 = df0.stack().reset_index().rename(columns = {'Alimento': 'alimento', 
                                                  'level_1': 'faixa etária', 0: 'preço'})
df0.set_index('alimento', inplace = True)
df0 = df0.sort_values(by = 'preço', ascending = False)
df0

,faixa etária,preço
alimento,,
Arroz,3 anos a 6 anos,"4,316.00"
Açucar,3 anos a 6 anos,"2,158.00"
Feijão,3 anos a 6 anos,"2,158.00"
Macarrão,3 anos a 6 anos,"2,158.00"
Massa de milho,3 anos a 6 anos,"2,158.00"
Sal,3 anos a 6 anos,"2,158.00"
Óleo,3 anos a 6 anos,"1,942.20"
Leite,3 anos a 6 anos,"1,726.40"
Biscoito cracker,3 anos a 6 anos,863.20


<div class="alert alert-block alert-info">
<b>Getting the mean values</b>
</div>

In [8]:
df1 = df.groupby(['Alimento', 'Faixa etária']).mean()
df1.head()

Quantidade de alunos  Quantidade/aluno  kg/unidade  \
Alimento Faixa etária                                                           
Arroz    3 anos a 6 anos                   2158                 2        1.00   
         6 anos a 15 anos                  9691                 2        1.00   
         6 meses a 3 anos                  1998                 1        1.00   
Açucar   3 anos a 6 anos                   2158                 1        1.00   
         6 anos a 15 anos                  9691                 1        1.00   

                           Preço/unidade  kg/aluno  Preço/aluno  kg total  \
Alimento Faixa etária                                                       
Arroz    3 anos a 6 anos            2.89      2.00         5.78  4,316.00   
         6 anos a 15 anos           2.89      2.00         5.78 19,382.00   
         6 meses a 3 anos           2.89      1.00         2.89  1,998.00   
Açucar   3 anos a 6 anos            2.29      1.00         2.29  2,158.00   
         6 anos a 15 anos           2.29      1.00         2.29  9,691.00   

                           Preço total  
Alimento Faixa etária                   
Arroz    3 anos a 6 anos     12,473.24  
         6 anos a 15 anos    56,013.98  
         6 meses a 3 anos     5,774.22  
Açucar   3 anos a 6 anos      4,941.82  
         6 anos a 15 anos    22,192.39

<div class="alert alert-block alert-info">
<b>Filtering the first pivot table</b>
</div>

In [9]:
# Can't get to filter tuples, axis = 1 (column) is the default
df2 = pivot_all.filter(items = ['Leite', 'Macarrão', 'Massa de milho'], axis = 0)
df2

kg total                                  
Faixa etária   3 anos a 6 anos 6 anos a 15 anos 6 meses a 3 anos
Alimento                                                        
Leite                 1,726.40             0.00         1,598.40
Macarrão              2,158.00         9,691.00           999.00
Massa de milho        2,158.00         9,691.00             0.00

<div class="alert alert-block alert-info">
<b>Another way to filter data</b>
</div>

In [10]:
# It's also possible to filter them
df_ini = df[df['Faixa etária'] == '6 meses a 3 anos']
df_mei = df[df['Faixa etária'] == '3 anos a 6 anos']
df_fin = df[df['Faixa etária'] == '6 anos a 15 anos']

<div class="alert alert-block alert-info">
<b>Examples</b>
</div>

In [11]:
df_mei.filter(items = ['Leite', 'Macarrão', 'Massa de milho'], axis = 0)

,Faixa etária,Quantidade de alunos,Quantidade/aluno,kg/unidade,Preço/unidade,kg/aluno,Preço/aluno,kg total,Preço total
Alimento,,,,,,,,,
Leite,3 anos a 6 anos,2158,4,0.20,4.69,0.80,18.76,"1,726.40","40,484.08"
Macarrão,3 anos a 6 anos,2158,2,0.50,1.99,1.00,3.98,"2,158.00","8,588.84"
Massa de milho,3 anos a 6 anos,2158,2,0.50,1.49,1.00,2.98,"2,158.00","6,430.84"


In [12]:
df_mei.filter(items = ['kg/unidade'], axis = 1)

,kg/unidade
Alimento,
Arroz,1.00
Açucar,1.00
Biscoito cracker,0.40
Biscoito maria,0.40
Feijão,1.00
Leite,0.20
Macarrão,0.50
Massa de milho,0.50
Sal,1.00


<div class="alert alert-block alert-info">
<b>Plot of alimentos by faixa etária</b>
</div>

In [13]:
fig = go.Figure()

fig.add_trace(go.Bar(x = df_ini.index, y = df_ini['Preço total'], 
    name = '6 meses a 3 anos',
    hovertemplate = '%{label}: <br>Preço: %{y}',
    marker_color = 'rgb(255, 255, 145)',
    marker_line_color = 'rgb(1,1,1)',
    marker_line_width = 2))

fig.add_trace(go.Bar(x = df_mei.index, y = df_mei['Preço total'], 
    name = '3 anos a 6 anos', 
    hovertemplate = '%{label}: <br>Preço: %{y}',
    marker_color = 'rgb(115, 150, 200)',
    marker_line_color = 'rgb(1,1,1)',
    marker_line_width = 2))

fig.add_trace(go.Bar(x = df_fin.index, y = df_fin['Preço total'],
    name = '6 anos a 15 anos',
    hovertemplate = "%{label}: <br>Preço: %{y}",
    marker_color ='rgb(190, 230, 160)',
    marker_line_color = 'rgb(1,1,1)',
    marker_line_width = 2))

fig.update_layout(barmode = 'stack', 
    title_text = 'Cestas por grupos', 
    title_x = 0.5,
    title_y = 0.9,
    legend = dict(
    x = 0, y = 1.5),
    legend_title='<b> Grupos </b>',
    xaxis_title = 'Alimentos',
    yaxis_title = 'Preço total',
    paper_bgcolor = 'rgba(1,1,1,1)',
    plot_bgcolor = 'rgba(1,1,1,1)',
    hoverlabel = dict(
    bgcolor = 'white',
    font_color = 'black',
    font_size = 15, 
    font_family = 'Helvetica'),
    font = dict(
    family = 'Helvetica, Helvetica',
    size = 15,
    color = 'white'))

fig.update_yaxes(hoverformat = '.2f', showgrid = False)
fig.update_xaxes(showgrid = False)

py.iplot(fig, filename = 'Cestas por grupo', auto_open = False)